In [ ]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Examine Train/Test Sets

In [187]:
df_train_sets = pd.read_pickle('ebw_poc_train_sample_h1.pkl')
df_test_sets  = pd.read_pickle('ebw_poc_test_sample_h1.pkl')

In [ ]:
pd.concat([df_train_sets.groupby(['weeks_since_hit'])['target'].count(), df_test_sets.groupby(['weeks_since_hit'])['target'].count()], axis=1)

In [ ]:
df_train_sets.groupby(['weeks_since_hit', 'censoring_flg_mod'])['target'].count()/df_train_sets.groupby(['weeks_since_hit'])['target'].count()

In [ ]:
df_extract_train = df_train_sets[df_train_sets.index.get_level_values(3) == 5]
df_extract_test =  df_test_sets[df_test_sets.index.get_level_values(3) == 5]

In [ ]:
df_extract_train.groupby(['weeks_since_hit', 'censoring_flg_mod'])['target'].count()

In [ ]:
df_extract_test.groupby(['weeks_since_hit', 'censoring_flg_mod'])['target'].count()

In [ ]:
df_extract_train['target'].plot(kind='hist')

In [ ]:

df_extract_train['target'].apply(lambda x: np.log(x)).plot(kind='hist')

## Model Training

In [188]:
# import sksurv and sklearn packages
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.ensemble import GradientBoostingSurvivalAnalysis, RandomSurvivalForest
from sksurv.metrics import as_concordance_index_ipcw_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import time
import pickle

#### define prereqs

In [189]:
id_cols = list(df_train_sets.index)
target_cols = ['censoring_flg_mod', 'target']
cat_cols = ['market_name']
feature_cols = list(set(df_train_sets.columns) - set(target_cols))
num_cols = list(set(feature_cols) -set(cat_cols))

In [ ]:
split_number = 5
random_state = 1000
weeks_range = range(27)

# hyperparam grid for ElasticNet Baseline
param_grid_cnet = {
    'estimator__l1_ratio': [0.5, 0.1, 0.01, 0.001]
}

# hyperparam grid for Gradient Boosted Model
param_grid_gbm = {
    'estimator__max_depth': [2, 4, 6, 8, 10],
    'estimator__max_features': ['auto', 'sqrt', 'log2', None],
    'estimator__n_estimators': [50, 100, 150, 200],
    'estimator__learning_rate': [0.01, 0.001, 0.5, 0.1],
    'estimator__loss': ['coxph'],
    'estimator__subsample': [0.70, 0.80, 0.9, 1.0],
    'estimator__min_samples_split': [2, 4, 8, 16],
    'estimator__min_samples_leaf': [2, 4, 8, 16]
}

# hyperparam grid for Random Survival Forest
param_grid_rsf = {
    'estimator__max_depth': [2, 4, 6, 8, 10],
    'estimator__max_features': ['auto', 'sqrt', 'log2', None],
    'estimator__n_estimators': [50, 100, 150, 200],
    'estimator__min_samples_split': [2, 4, 8, 16, 32],
    'estimator__min_samples_leaf': [2, 4, 8, 16, 32],
    'estimator__max_leaf_nodes': [2, 4, 8, 16, 32],
    'estimator__bootstrap': [True, False],
    'estimator__oob_score': [False, True],
    'estimator__max_samples': [0.70, 0.80, 0.90, 1.0]
}

# best scores - train/test
best_scores_cnet_train = {}
best_scores_gbm_train = {}
best_scores_rsf_train = {}

best_scores_cnet_test = {}
best_scores_gbm_test = {}
best_scores_rsf_test = {}

# best estimators
best_estimators_cnet = {}
best_estimators_gbm = {}
best_estimators_rsf = {}

# pickle file names
file_cnet='best_est_cnet_h1_20230630.pkl'
file_gbm='best_est_gbm_h1_20230630.pkl'
file_rsf='best_est_rsf_h1_20230630.pkl'

#### train a baseline ElasticNet model

In [ ]:
for wk in weeks_range:
    start = time.perf_counter()

    sv = CoxnetSurvivalAnalysis()

    X_train = df_train_sets[df_train_sets.index.get_level_values(3) == wk][feature_cols]
    X_train = pd.get_dummies(X_train, columns=['market_name'])
    X_train[num_cols] = StandardScaler().fit_transform(X_train[num_cols])
    y_train = df_train_sets[df_train_sets.index.get_level_values(3) == wk][target_cols].to_records(index=False)

    sv_model = RandomizedSearchCV(estimator=as_concordance_index_ipcw_scorer(sv), param_distributions=param_grid_cnet,
                                  cv=split_number, verbose=1, n_iter=50, n_jobs=-1)
    sv_model.fit(X_train, y_train)
    best_estimators_cnet[wk] = sv_model.best_estimator_
    stop = time.perf_counter()
    print(str(wk) + '_' + 'best_score: ' + f'{sv_model.best_score_}')
    print('Elapsed time: '  + str(int((stop-start))))

In [ ]:
# write best estimators to a pickle file
with open(file_cnet, 'wb') as file:
    pickle.dump(best_estimators_cnet, file)

### train a GBM model

In [ ]:
# train a gbm
for wk in weeks_range:
    start = time.perf_counter()

    sv = GradientBoostingSurvivalAnalysis()

    X_train = df_train_sets[df_train_sets.index.get_level_values(3) == wk][feature_cols]
    X_train = pd.get_dummies(X_train, columns=['market_name'])
    df_y_temp = df_train_sets[df_train_sets.index.get_level_values(3) == wk][target_cols]
    y_train = df_y_temp.to_records(index=False)

    # train gbm model
    sv_model = RandomizedSearchCV(estimator=as_concordance_index_ipcw_scorer(sv), param_distributions=param_grid_gbm,
                                  cv=split_number, verbose=1, n_iter=50, n_jobs=-1)
    sv_model.fit(X_train, y_train)
    best_estimators_gbm[wk] = sv_model.best_estimator_
    stop = time.perf_counter()
    print(str(wk) + '_' + 'best_score: ' + f'{sv_model.best_score_}')
    print('Elapsed time: '  + str(int((stop-start))))

In [ ]:
# write best estimators to a pickle file
with open(file_gbm, 'wb') as file:
    pickle.dump(best_estimators_gbm, file)

#### train a RandomSurvivalForest

In [ ]:
# train a gbm
for wk in weeks_range:
    start = time.perf_counter()

    sv = RandomSurvivalForest()

    X_train = df_train_sets[df_train_sets.index.get_level_values(3) == wk][feature_cols]
    X_train = pd.get_dummies(X_train, columns=['market_name'])
    df_y_temp = df_train_sets[df_train_sets.index.get_level_values(3) == wk][target_cols]
    y_train = df_y_temp.to_records(index=False)

    # train rsf model
    sv_model = RandomizedSearchCV(estimator=as_concordance_index_ipcw_scorer(sv), param_distributions=param_grid_rsf,
                                  cv=split_number, verbose=1, n_iter=50, n_jobs=-1)
    sv_model.fit(X_train, y_train)
    best_estimators_rsf[wk] = sv_model.best_estimator_
    stop = time.perf_counter()
    print(str(wk) + '_' + 'best_score: ' + f'{sv_model.best_score_}')
    print('Elapsed time: '  + str(int((stop-start))))

In [ ]:
# write best estimators to a pickle file
with open(file_rsf, 'wb') as file:
    pickle.dump(best_estimators_rsf, file)